## Imports

In [64]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd

import numpy as np

## Set up API connection with Spotipy

In [65]:
api_codes = pd.read_csv('API_codes.csv').loc[0]

In [66]:
client_id = api_codes['client_id']
client_secret = api_codes['client_secret']

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id, client_secret))

In [67]:
# this type of authentication is not used here
scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id, client_secret, redirect_uri='http://localhost', scope=scope))

## Get list of all playlists

In [68]:
my_username = 'duckyninjamath'

In [69]:
my_playlists = spotify.user_playlists(my_username)['items']
my_playlists += spotify.user_playlists(my_username, offset=50)['items']
my_playlists += spotify.user_playlists(my_username, offset=100)['items']

In [70]:
playlists_df_raw = pd.DataFrame.from_dict(my_playlists).reset_index().set_index('name')

playlists_df_raw['num_tracks'] = playlists_df_raw.tracks.apply(lambda d: d['total'])

playlists_df_raw.sort_values('num_tracks', ascending=False)

,index,collaborative,description,external_urls,href,id,images,owner,primary_color,public,snapshot_id,tracks,type,uri,num_tracks
name,,,,,,,,,,,,,,,
All Music before Mission,91,False,,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/4mmuHrjeU...,4mmuHrjeU38p0K6M8pC73w,"[{'height': 640, 'url': 'https://mosaic.scdn.c...","{'display_name': 'Neil Thompson', 'external_ur...",None,True,MTIsODYwZmMwYTkwMzUxYmI1NjQzNTIxZDIxNjk1MDkyN2...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:4mmuHrjeU38p0K6M8pC73w,941
Focus,45,False,,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/45JTnzWMn...,45JTnzWMn7TW1VJW69Wl6T,"[{'height': 640, 'url': 'https://mosaic.scdn.c...","{'display_name': 'Neil Thompson', 'external_ur...",None,True,MTEzLDk4NjFkMGE1Yzg1ZTliM2Q0ZWU5NjI2ZjlhZjAyND...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:45JTnzWMn7TW1VJW69Wl6T,762
Funky/Complex,112,False,It&#x27;s exactly what it sounds like.,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/1iY7yiUEr...,1iY7yiUErGixFgQYfMkfcr,"[{'height': None, 'url': 'https://i.scdn.co/im...","{'display_name': 'Michael DeBrota', 'external_...",None,True,MzI2LDJiNGNlMmQxZGYyODdhNDIxYzZmNzQyOWUwOWJhMz...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:1iY7yiUErGixFgQYfMkfcr,330
Good dnb,43,False,The most important playlist,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/3eg1TQTtl...,3eg1TQTtluTlwRP2leILsG,"[{'height': 640, 'url': 'https://mosaic.scdn.c...","{'display_name': 'Neil Thompson', 'external_ur...",None,True,MjA3LDE0NzYxZGFiNGE3YWVhMDA2MWNkYzFiNWZkMDQyZW...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:3eg1TQTtluTlwRP2leILsG,196
Great Music,12,False,,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/3WSMk1vHO...,3WSMk1vHOc59SsoDHp3JB9,"[{'height': 640, 'url': 'https://mosaic.scdn.c...","{'display_name': 'Neil Thompson', 'external_ur...",None,True,MTA1LDYxOTM4N2UzNDI4NzBmNmU2NWFmNDYxNWVjMDg3ZT...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:3WSMk1vHOc59SsoDHp3JB9,144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Groove,42,False,,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/3kmzNIh5N...,3kmzNIh5NPXkKn10CYuuU1,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","{'display_name': 'Neil Thompson', 'external_ur...",None,True,MixmNTljYTQwZDcwNmFlODBlY2ZhNTg5MjQ2ODExNWY2YT...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:3kmzNIh5NPXkKn10CYuuU1,1
Post Mission Jams,89,False,,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/05dR3xh32...,05dR3xh328nkeAWJTIwYDo,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","{'display_name': 'Neil Thompson', 'external_ur...",None,True,NSw3YmNkNmY3ODA1NTBlZjhmYzBjNTExMzdhNDBjMGI1Y2...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:05dR3xh328nkeAWJTIwYDo,1
Doing the Things,55,False,,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/4ChSsWxZf...,4ChSsWxZfNoIKqZxiVmWz8,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","{'display_name': 'Neil Thompson', 'external_ur...",None,True,MyxlMDIyMDllYmJkZjdmOGU3YjdhOWIyODY0ZWVkZGMxY2...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:4ChSsWxZfNoIKqZxiVmWz8,1


## Find playlist called 'Focus', which has 762 songs at the time of running

In [71]:
focus_playlist_index = playlists_df_raw[playlists_df_raw.index == 'Focus']['index'][0]

In [72]:
focus_playlist_uri = my_playlists[focus_playlist_index]['uri']

In [73]:
focus_songs = {}
for run in range(8):
    for i, song in enumerate(spotify.playlist_items(focus_playlist_uri, offset=run*100)['items']):
        track = song['track']
        focus_songs[i + run*100] = track

## Build pandas df with this playlist

In [54]:
focus_df = pd.DataFrame.from_dict(focus_songs, orient='index').reset_index().set_index('name')

#### Add and remove features to get clean dataset

In [74]:
focus_df['Principle_Artist'] = focus_df.artists.apply(lambda l: l[0]['name'])

focus_df['Secondary_Artist'] = focus_df.artists.apply(lambda l: l[1]['name'] if len(l) > 1 else np.NaN)

focus_df['Duration_total_seconds'] = focus_df.duration_ms.apply(lambda t: t//1000) 

focus_df['Duration_minutes_part'] = focus_df.duration_ms.apply(lambda t: t//1000//60)

focus_df['Duration_seconds_part'] = focus_df.duration_ms.apply(lambda t: t//1000%60) 

focus_df['Album_name'] = focus_df.album.apply(lambda a: a['name'])

focus_df['Album_release_date'] = focus_df.album.apply(lambda a: a['release_date'])

In [75]:
focus_df_final = focus_df[['Album_name', 'Principle_Artist', 'Secondary_Artist',
       'Album_release_date', 'Duration_total_seconds', 'Duration_minutes_part',
       'Duration_seconds_part', 'popularity', 'index']]

In [76]:
focus_df_final

,Album_name,Principle_Artist,Secondary_Artist,Album_release_date,Duration_total_seconds,Duration_minutes_part,Duration_seconds_part,popularity,index
name,,,,,,,,,
Golden Light,Nocturne Ep,Keeno,NaN,2013-12-09,449,7,29,0,0
Voyager - Original,Voyager,Betoko,NaN,2010-01-04,493,8,13,1,1
From Home,Past Is Prologue,Tycho,NaN,2006,391,6,31,0,2
Sunrise Projector,Past Is Prologue,Tycho,NaN,2006,184,3,4,0,3
Dictaphone’s Lament,Past Is Prologue,Tycho,NaN,2006,310,5,10,0,4
...,...,...,...,...,...,...,...,...,...
Crescent Spoon,Crescent Spoon,Regressor,Arenlong,2022-07-15,453,7,33,21,757
Kilauea,Kilauea,Bensley,NaN,2019-04-19,323,5,23,5,758
Cryogen,Cryogen,daniel hayn,Israel Strom,2022-03-25,202,3,22,28,759
